### Part 1.1  基于枚举方法来搭建中文分词工具（旧）

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = "./data/综合类中文词库.xlsx"
data_frame = pd.read_excel(path, header = None)

In [ ]:
dic_word_list = data_frame[data_frame.columns[0]].tolist()

In [ ]:
dic_words =  dic_word_list   # 保存词典库中读取的单词

# 以下是每一个单词出现的概率。为了问题的简化，我们只列出了一小部分单词的概率。 在这里没有出现的的单词但是出现在词典里的，统一把概率设置成为0.00001
# 比如 p("学院")=p("概率")=...0.00001

word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}

print (sum(word_prob.values()))

In [ ]:
for key, value in word_prob.items():
    if key not in dic_words:
        dic_words.append(key)

In [ ]:
for item in dic_words:
    word_prob.setdefault(item, 0.00001)

In [ ]:
print (sum(word_prob.values()))

In [ ]:
def split_word_with_dic_front_max(dic=[], input_str=""):
    '''前项最大分词'''
    input_str_tmp = input_str
    segments = []
    while(input_str_tmp!=""):
        tmp_length = 0
        tmp_dic_content = ""
        for word in dic:
            # print("单词长度："+str(len(word))+" === 单词："+word +"==""=== 划分："+input_str_tmp[:len(word)])
            if(word == input_str_tmp[:len(word)]):
                if(tmp_length < len(word)):
                    tmp_dic_content = word
                    tmp_length = len(word)
        segments.append(tmp_dic_content)
        # print(tmp_dic_content)
        input_str_tmp = input_str_tmp[tmp_length:]
    return segments

In [ ]:
def split_word_with_dic_front_min(dic=[], input_str=""):
    '''前项最小分词'''
    input_str_tmp = input_str
    segments = []
    while(input_str_tmp!=""):
        tmp_length = len(input_str)
        tmp_dic_content = ""
        for word in dic:
            # print("单词长度："+str(len(word))+" === 单词："+word +"==""=== 划分："+input_str_tmp[:len(word)])
            if(word == input_str_tmp[:len(word)]):
                if(tmp_length > len(word)):
                    tmp_dic_content = word
                    tmp_length = len(word)
        segments.append(tmp_dic_content)
        # print(tmp_dic_content)
        input_str_tmp = input_str_tmp[tmp_length:]
    return segments

In [ ]:
def split_word_with_dic_back_max(dic=[], input_str=""):
    '''后项最大分词'''
    input_str_tmp = input_str
    segments = []
    while(input_str_tmp!=""):
        tmp_length = 0
        tmp_dic_content = ""
        for word in dic:
            if(word == input_str_tmp[-len(word):]):
                if(tmp_length < len(word)):
                    tmp_dic_content = word
                    tmp_length = len(word)
        segments.append(tmp_dic_content)
        input_str_tmp = input_str_tmp[:-tmp_length]
    return segments[::-1]

In [ ]:
def split_word_with_dic_back_min(dic=[], input_str=""):
    '''后项最小分词'''
    input_str_tmp = input_str
    segments = []
    while(input_str_tmp!=""):
        tmp_length = len(input_str)
        tmp_dic_content = ""
        for word in dic:
            if(word == input_str_tmp[-len(word):]):
                if(tmp_length > len(word)):
                    tmp_dic_content = word
                    tmp_length = len(word)
        segments.append(tmp_dic_content)
        input_str_tmp = input_str_tmp[:-tmp_length]
    return segments[::-1]


In [ ]:
def split_word(dic=[], input=""):
    tmp_result = []
    tmp_result.append(split_word_with_dic_front_max(dic, input))
    tmp_result.append(split_word_with_dic_back_max(dic, input))    
    tmp_result.append(split_word_with_dic_front_min(dic, input))
    tmp_result.append(split_word_with_dic_back_min(dic, input))     
    return tmp_result

In [ ]:
def get_split_probability_use_segment(split_word_segment=[]):
    sum_result = 0
    for seg in split_word_segment:
        sum_result -= np.log(word_prob.get(seg))
    return sum_result
def get_split_probability(split_word_segments=[[], ]):
    '''
    根据传入的分词结果计算出概率最高的分词结果并返回
    '''
    index = 0
    max_index = 0
    sum = get_split_probability_use_segment(split_word_segments[0])
    for segment in split_word_segments:
        tmp = get_split_probability_use_segment(segment)
        if(sum>tmp): 
            sum = tmp
            max_index = index
        index += 1
    return split_word_segments[max_index], sum

In [ ]:
#  分数（10）
## TODO 请编写word_segment_naive函数来实现对输入字符串的分词
def word_segment_naive(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的最作为最后结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """

    # TODO： 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。 
    segments = split_word(dic_words, input_str)  # 存储所有分词的结果。如果次字符串不可能被完全切分，则返回空列表(list)
                   # 格式为：segments = [["今天"，“天气”，“好”],["今天"，“天“，”气”，“好”],["今“，”天"，“天气”，“好”],...]
    
    # TODO: 第二步：循环所有的分词结果，并计算出概率最高的分词结果，并返回
    best_segment, best_score = get_split_probability(segments)
    
    return best_segment      

In [ ]:
# 测试
print(word_segment_naive("北京的天气真好啊"))
print(word_segment_naive("今天的课程内容很有意思"))
print(word_segment_naive("经常有意见分歧"))